In [103]:
import pandas as pd
# Reset the options to default
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 500)

# Set the maximum number of rows to display
pd.set_option('display.max_rows', None)

In [104]:
hprc_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/data/production/hprc-production-biosample-table.tsv', sep='\t')
hprc_biosample.rename(columns={'Sample':'sample_ID'}, inplace=True)

In [105]:
wustl_y1_topup_sra = pd.read_csv('HPRC_WUSTL_Y1_PacBio_HiFi_TopUp_Metadata_Submission_v0.1.tsv',sep='\t')
wustl_y1_topup_sra['sample_id'] = wustl_y1_topup_sra['sample_ID'] + '-' + wustl_y1_topup_sra['filename']
wustl_y1_topup_sra['sample_id'] = [sample.split('.')[0] for sample in wustl_y1_topup_sra['sample_id'].tolist()]

In [106]:
# List of column names from your metadata sheet
metadata_columns = [
    'filename', 'library_ID', 'library_strategy', 'library_source', 
    'library_selection', 'library_layout', 'platform', 'instrument_model', 
    'design_description', 'data_type', 'shear_method', 'size_selection', 
    'ccs_algorithm', 'polymerase_version', 'seq_plate_chemistry_version', 
    'generator_facility', 'generator_contact', 'biosample_accession', 
    'title', 'filetype', 'assembly'
]

# Assume you have another DataFrame named 'other_df'
# Example: other_df = pd.read_csv('path_to_other_file.csv')

# Check if the metadata columns are present in the other DataFrame
missing_columns = [col for col in metadata_columns if col not in wustl_y1_topup_sra.columns]

# Output the missing columns
print("Missing columns:", missing_columns)


Missing columns: ['biosample_accession', 'title', 'assembly']


In [107]:
wustl_y1_topup_output_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/WUSTL_HPRC_HiFi_Year1_TopUp/hifi_qc/hifi_qc_samples_output_table.csv')

In [108]:
wustl_y1_topup_sra = wustl_y1_topup_sra[wustl_y1_topup_sra['sample_id'].isin(wustl_y1_topup_output_df['sample_id'].tolist())]

In [109]:
assert wustl_y1_topup_output_df.shape[0] == wustl_y1_topup_sra.shape[0]

In [110]:
wustl_y1_topup_sra['assembly'] = 'unaligned'

In [111]:
wustl_y1_topup_sra = pd.merge(hprc_biosample, wustl_y1_topup_sra, on='sample_ID', how='inner')

In [112]:
wustl_y1_topup_sra['title'] = 'PacBio HiFi sequencing of ' + wustl_y1_topup_sra['sample_ID']

In [113]:
wustl_y1_topup_sra.rename(columns={'Accession':'biosample_accession'},inplace=True)

In [114]:
wustl_y1_topup_sra = wustl_y1_topup_sra[metadata_columns]

In [115]:
wustl_y1_topup_sra['instrument_model'] = 'Revio'

In [116]:
wustl_y1_topup_sra['library_ID'] = wustl_y1_topup_sra['library_ID'] + '_' + wustl_y1_topup_sra['filename']

In [117]:
wustl_y1_topup_sra['library_ID'] = [library_id.split('.')[0] for library_id in wustl_y1_topup_sra['library_ID'].tolist()]

In [118]:
wustl_y1_topup_sra['library_layout'] = 'single'

In [119]:
wustl_y1_topup_sra.to_csv('sra_biosample_WUSTL_Y1_TopUp.tsv',index=False, sep='\t')